In [1]:
import os
import torch
from torch import nn

torch.__version__

'2.0.0'

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
import os
import zipfile

from pathlib import Path
import random
import shutil
from shutil import copyfile

import requests
import pandas as pd


data_path = Path("/kaggle/input/ham10000-splitted/Images splitted/Splitted")
image_path = data_path # FOLDER path = splitted dataset into Train & Test

In [6]:
# no of samples per class

classes = os.listdir(f'{image_path}/train')
classes_no_of_data_dict = {}
# classes.remove("desktop.ini")
classes

for i in range(len(classes)):
    classes_no_of_data_dict[classes[i]] = len(os.listdir(f'{image_path}/train/{classes[i]}'))
    
# for i in range(len(classes)):
#     temp = len(os.listdir(f'{image_path}/test/{classes[i]}'))
#     classes_no_of_data_dict[classes[i]] = classes_no_of_data_dict[classes[i]] + temp

classes_no_of_data_dict

{'mel': 891,
 'vasc': 114,
 'df': 93,
 'nv': 5365,
 'bkl': 880,
 'akiec': 262,
 'bcc': 412}

In [15]:
# Setup train and testing paths
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(PosixPath('/kaggle/input/ham10000-splitted/Images splitted/Splitted/train'),
 PosixPath('/kaggle/input/ham10000-splitted/Images splitted/Splitted/test'))

In [16]:
from torchvision import datasets, transforms
from torchvision.transforms.functional import InterpolationMode


IMG_SIZE = 450


# Create transform pipeline manually   
data_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomPerspective(distortion_scale=0.2),
    
    # Calculated for train data
    transforms.Normalize([0.5018, 0.5015, 0.5013], [0.1029, 0.0985, 0.0807]),
])     

print(f"Manually created transforms: {data_transform}")





data_transform_test = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),

    # Calculated for test data
    transforms.Normalize([0.5018, 0.5015, 0.5013], [0.1029, 0.0985, 0.0807]),
])  



train_data = datasets.ImageFolder(root=train_dir, 
                                  transform=data_transform, 
                                  target_transform=None) 


test_data = datasets.ImageFolder(root=test_dir,
                                 transform=data_transform_test,
                                )

print(f"Train data:\n{train_data}\nTest data:\n{test_data}")

Manually created transforms: Compose(
    Resize(size=(450, 450), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    RandomHorizontalFlip(p=0.5)
    RandomVerticalFlip(p=0.5)
    RandomPerspective(p=0.5)
    Normalize(mean=[0.5018, 0.5015, 0.5013], std=[0.1029, 0.0985, 0.0807])
)
Train data:
Dataset ImageFolder
    Number of datapoints: 8017
    Root location: /kaggle/input/ham10000-splitted/Images splitted/Splitted/train
    StandardTransform
Transform: Compose(
               Resize(size=(450, 450), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               RandomHorizontalFlip(p=0.5)
               RandomVerticalFlip(p=0.5)
               RandomPerspective(p=0.5)
               Normalize(mean=[0.5018, 0.5015, 0.5013], std=[0.1029, 0.0985, 0.0807])
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 1998
    Root location: /kaggle/input/ham10000-splitted/Images splitted/Splitted/test
    StandardTransform
T

In [17]:
import torchvision

weights = torchvision.models.ResNeXt50_32X4D_Weights.DEFAULT 
weights

ResNeXt50_32X4D_Weights.IMAGENET1K_V2

In [18]:
auto_transforms = weights.transforms()
auto_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [20]:
# Get class names as a list
class_names = train_data.classes
class_names

['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [21]:
# Can also get class names as a dict
class_dict = train_data.class_to_idx
class_dict

{'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}

In [23]:
# Check the lengths
len(train_data), len(test_data)

(8017, 1998)

In [25]:
from torch.utils.data import DataLoader, WeightedRandomSampler

BATCH_SIZE = 32

train_dataloader = DataLoader(dataset=train_data, 
                              batch_size=BATCH_SIZE, 
                              num_workers=4, 
                              shuffle=True,
                              pin_memory=True,
                              ) 



test_dataloader = DataLoader(dataset=test_data, 
                             batch_size=BATCH_SIZE, 
                             num_workers=4, 
                             shuffle=False,
                             pin_memory=True,
                             ) 

len(train_dataloader), len(test_dataloader)

(251, 63)

In [27]:
img, label = next(iter(train_dataloader))

print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}")

Image shape: torch.Size([32, 3, 450, 450]) -> [batch_size, color_channels, height, width]
Label shape: torch.Size([32])


In [36]:
image_batch, label_batch = next(iter(train_dataloader))
image, label = image_batch[0], label_batch[0]
image.shape, label

(torch.Size([3, 450, 450]), tensor(5))

In [37]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sequential as Seq

import sys
sys.path.append("/kaggle/input/vig-pytorch/")
from gcn_lib import Grapher, act_layer


from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.models.helpers import load_pretrained
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
from timm.models.registry import register_model

In [38]:
model = torchvision.models.resnext50_32x4d(weights=weights).to(device)
# model


Downloading: "https://download.pytorch.org/models/resnext50_32x4d-1a0047aa.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-1a0047aa.pth
100%|██████████| 95.8M/95.8M [00:01<00:00, 81.7MB/s]


In [39]:
from torchinfo import summary


model_summary = model


summary(model=model_summary,
        input_size=(16, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [16, 3, 224, 224]    [16, 1000]           --                   True
├─Conv2d (conv1)                         [16, 3, 224, 224]    [16, 64, 112, 112]   9,408                True
├─BatchNorm2d (bn1)                      [16, 64, 112, 112]   [16, 64, 112, 112]   128                  True
├─ReLU (relu)                            [16, 64, 112, 112]   [16, 64, 112, 112]   --                   --
├─MaxPool2d (maxpool)                    [16, 64, 112, 112]   [16, 64, 56, 56]     --                   --
├─Sequential (layer1)                    [16, 64, 56, 56]     [16, 256, 56, 56]    --                   True
│    └─Bottleneck (0)                    [16, 64, 56, 56]     [16, 256, 56, 56]    --                   True
│    │    └─Conv2d (conv1)               [16, 64, 56, 56]     [16, 128, 56, 56]    8,192                True
│    │    └─BatchN

In [41]:
# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)


output_shape = len(class_names)


model.fc = torch.nn.Linear(in_features=2048, 
                    out_features=output_shape, 
                    bias=True).to(device)

In [42]:
summary(model=model_summary,
        input_size=(16, 3, 224, 224), # (batch_size, num_patches, embedding_dimension)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [16, 3, 224, 224]    [16, 7]              --                   True
├─Conv2d (conv1)                         [16, 3, 224, 224]    [16, 64, 112, 112]   9,408                True
├─BatchNorm2d (bn1)                      [16, 64, 112, 112]   [16, 64, 112, 112]   128                  True
├─ReLU (relu)                            [16, 64, 112, 112]   [16, 64, 112, 112]   --                   --
├─MaxPool2d (maxpool)                    [16, 64, 112, 112]   [16, 64, 56, 56]     --                   --
├─Sequential (layer1)                    [16, 64, 56, 56]     [16, 256, 56, 56]    --                   True
│    └─Bottleneck (0)                    [16, 64, 56, 56]     [16, 256, 56, 56]    --                   True
│    │    └─Conv2d (conv1)               [16, 64, 56, 56]     [16, 128, 56, 56]    8,192                True
│    │    └─BatchN

In [43]:
model = nn.DataParallel(model)  ### for two GPU faster computations 

In [45]:
import wandb

# start a new wandb run to track this script
tracking = wandb.init(
    # set the wandb project where this run will be logged
    project="Glaucoma Fundus Imaging",
    name="HAM10000: ResNext50",
#     name="Origa: Fb_model",
    notes="",
    # track hyperparameters and run metadata
    config={
        "learning_rate": 2e-3,
        "architecture": "fb",
        "dataset": "Default",
        "epochs": 200,
    }
)

# 4b7dfb240ea0d5aae1afac2de518c0e940547396

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [46]:
"""
Contains functions for training and testing a PyTorch model.
"""
from typing import Dict, List, Tuple
from sklearn.metrics import classification_report

import torch

from tqdm.auto import tqdm
import wandb


def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device,
               len_train_data) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch.

    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).

    Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
    """
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss = 0.0 
    train_acc = 0.0

    # for evaluation
    y_true_train_data = []
    y_predicted_train_data = []

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)


        # 1. Forward pass
        y_pred = model(X)
        
#         y_pred = y_pred.logits

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        # train_acc += (y_pred_class == y).sum().item()/len(y_pred)
        train_acc += (y_pred_class == y).sum().item()
        
        # for evaluaiton
        y_true_train_data.append(y)
        y_predicted_train_data.append(y_pred_class)

    # Adjust metrics to get average loss and accuracy per batch 
    train_loss = train_loss / len(dataloader)
#     train_acc = train_acc / len(dataloader)
    train_acc = train_acc / len_train_data
    
    #     New way of accuracy calculation
    true = 0
    tot = 0
    for i in range(len(y_true_train_data)):
        x = (y_true_train_data[i] == y_predicted_train_data[i])
        for j in range(len(x)):
            tot += 1
            if x[j] == True:
                true += 1
    train_acc = (true/tot)

    # return train_loss, train_acc
    return train_loss, train_acc, y_true_train_data, y_predicted_train_data




def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device,
              len_test_data) -> Tuple[float, float]:
    """Tests a PyTorch model for a single epoch.

    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.

    Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
    """
    # Put model in eval mode
    model.eval() 

    # Setup test loss and test accuracy values
    test_loss = 0.0 
    test_acc = 0.0

    # for evaluation
    y_true_test_data = []
    y_predicted_test_data = []


    # Turn on inference context manager
    with torch.inference_mode():

        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)
            
#             test_pred_logits = test_pred_logits.logits
            
            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            # test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
            test_acc += (test_pred_labels == y).sum().item()

            # test_pred_labels = torch.argmax(torch.softmax(test_pred_logits, dim=1), dim=1)
            # test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels)) 
            

            # for evaluaiton
            y_true_test_data.append(y)
            y_predicted_test_data.append(test_pred_labels)


    # Adjust metrics to get average loss and accuracy per batch 
    test_loss = test_loss / len(dataloader)
#     test_acc = test_acc / len(dataloader)
    test_acc = test_acc / len_test_data


    
#     New way of accuracy calculation
    true = 0
    tot = 0
    for i in range(len(y_true_test_data)):
        x = (y_true_test_data[i] == y_predicted_test_data[i])
        for j in range(len(x)):
            tot += 1
            if x[j] == True:
                true += 1
    test_acc = (true/tot)
    
    # return test_loss, test_acc
    return test_loss, test_acc, y_true_test_data, y_predicted_test_data




def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device,
          tracking,
          best_accuracy,
          len_train_data,
          len_test_data,
        ) -> Dict[str, List[float]]:
    """Trains and tests a PyTorch model.

    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.

    Calculates, prints and stores evaluation metrics throughout.

    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for 
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]} 
    For example if training for epochs=2: 
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]} 
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }


        
    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc, y_true_train_data, y_predicted_train_data = train_step(model=model,
                                                                                      dataloader=train_dataloader,
                                                                                      loss_fn=loss_fn,
                                                                                      optimizer=optimizer,
                                                                                      device=device,
                                                                                      len_train_data=len_train_data,
                                                                                      )
        test_loss, test_acc, y_true_test_data, y_predicted_test_data = test_step(model=model,
                                                                                 dataloader=test_dataloader,
                                                                                 loss_fn=loss_fn,
                                                                                 device=device,
                                                                                 len_test_data=len_test_data,
                                                                                 )

        if best_accuracy < test_acc:
            print(f"\nAccuracy Improved ({best_accuracy} to {test_acc}), Saving the model...............................................\n")
            best_accuracy = test_acc
            best_y_true_test_data = y_true_test_data
            best_y_predicted_test_data = y_predicted_test_data

            model_name_path = f"{best_accuracy} acc.pth"
            torch.save(model.state_dict(), model_name_path)
            
            
                        
            # taking "y_predicted_train_data" & "y_true_train_data" into 1D array because it came out as batch by batch 2D list
            predicted_train_data_1D = []
            true_train_data_1D = []

            for i in range(len(y_predicted_train_data)):
                for j in range(len(y_predicted_train_data[i])):
                    predicted_train_data_1D.append(y_predicted_train_data[i][j])
                    true_train_data_1D.append(y_true_train_data[i][j])

            # taking both into CPU
            predicted_train_data_cpu = torch.tensor(predicted_train_data_1D, device = 'cpu')
            true_train_data_cpu = torch.tensor(true_train_data_1D, device = 'cpu')



            # now same procedure for test data's
            predicted_test_data_1D = []
            true_test_data_1D = []

            for i in range(len(y_predicted_test_data)):
                for j in range(len(y_predicted_test_data[i])):
                    predicted_test_data_1D.append(y_predicted_test_data[i][j])
                    true_test_data_1D.append(y_true_test_data[i][j])

            # taking both into CPU
            predicted_test_data_cpu = torch.tensor(predicted_test_data_1D, device = 'cpu')
            true_test_data_cpu = torch.tensor(true_test_data_1D, device = 'cpu')



            # now same procedure for best test data's
            best_predicted_test_data_1D = []
            best_true_test_data_1D = []

            for i in range(len(best_y_predicted_test_data)):
                for j in range(len(best_y_predicted_test_data[i])):
                    best_predicted_test_data_1D.append(best_y_predicted_test_data[i][j])
                    best_true_test_data_1D.append(best_y_true_test_data[i][j])

            # taking both into CPU
            best_predicted_test_data_cpu = torch.tensor(best_predicted_test_data_1D, device = 'cpu')
            best_true_test_data_cpu = torch.tensor(best_true_test_data_1D, device = 'cpu')
            
            

            # Generate a classification report
            # F1 score on train data
            y_true_train = true_train_data_cpu
            y_pred_train = predicted_train_data_cpu

            report = classification_report(y_true_train, y_pred_train, target_names=class_names)

            print(f"Evaluation report on Train data: \n\n{report}\n\n\n\n")


            # F1 score on test data
            y_true_test = true_test_data_cpu
            y_pred_test = predicted_test_data_cpu

            report = classification_report(y_true_test, y_pred_test, target_names=class_names)

            print(f"Evaluation report on Test data: \n\n{report}")
            
            
            
            
            
            
        #     Live result tracking#####################################
        #     for i in range(len(y_true_test_data)):
        #         print(f"{y_true_test_data[i]} and {y_predicted_test_data[i]}")
        acc_0 = 0
        acc_1 = 0
        total_0 = 0
        total_1 = 0
        for i in range(len(y_predicted_test_data)):
            for j in range(len(y_true_test_data[i])):
                if (y_true_test_data[i][j] == 0):
                    total_0 += 1
                    if (y_true_test_data[i][j] == y_predicted_test_data[i][j]):
                        acc_0 += 1
                elif (y_true_test_data[i][j] == 1): 
                    total_1 += 1
                    if (y_true_test_data[i][j] == y_predicted_test_data[i][j]):
                        acc_1 += 1
#         print(f"\n0: {acc_0}/{total_0} and 1: {acc_1}/{total_1}\n\n")
        #     Live result tracking#####################################

        
        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f} | "
#           f"0: {acc_0}/{total_0} | "
#           f"1: {acc_1}/{total_1}"
        )
        

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        
        tracking.log({
            "train_acc": train_acc,
            "test_acc": test_acc,
        })

    # Return the filled results at the end of the epochs
    # return results
    

    return results, y_true_train_data, y_predicted_train_data, y_true_test_data, y_predicted_test_data, best_accuracy, best_y_true_test_data, best_y_predicted_test_data


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [47]:
from going_modular import engine

# Setup the optimizer to optimize our ViT model parameters using hyperparameters from the ViT paper 
optimizer = torch.optim.AdamW(params=model.parameters(), 
#                               lr=2e-3, # Base LR from Table 3 for ViT-* ImageNet-1k                                    ## learning rate decre..
                              lr = 1e-3,
                              betas=(0.9, 0.999), # default values but also mentioned in ViT paper section 4.1 (Training & Fine-tuning)
                              weight_decay=0.05,
#                               weight_decay=1e-5
                              ) # from the ViT paper section 4.1 (Training & Fine-tuning) and Table 3 for ViT-* ImageNet-1k

# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=5)
# Setup the loss function for multi-class classification
loss_fn = torch.nn.CrossEntropyLoss()

# Set the seeds
# set_seeds()
torch.manual_seed(42)
torch.cuda.manual_seed(42)

best_accuracy=0.0

In [ ]:
# Train the model and save the training results to a dictionary
results, y_true_train_data, y_predicted_train_data, y_true_test_data, y_predicted_test_data, best_accuracy, best_y_true_test_data, best_y_predicted_test_data = train(model=model,
                                                                                                           train_dataloader=train_dataloader,
                                                                                                           test_dataloader=test_dataloader,
                                                                                                           optimizer=optimizer,
                                                                                                           loss_fn=loss_fn,
                                                                                                           epochs=300,
                                                                                                           device=device,
                                                                                                           tracking=tracking,
                                                                                                           best_accuracy=best_accuracy,
                                                                                                           len_train_data=len(train_data),
                                                                                                           len_test_data=len(test_data)
                                                                                                           )

  0%|          | 0/300 [00:00<?, ?it/s]


Accuracy Improved (0.0 to 0.7582582582582582), Saving the model...............................................

Evaluation report on Train data: 

              precision    recall  f1-score   support

       akiec       0.45      0.28      0.35       262
         bcc       0.44      0.38      0.41       412
         bkl       0.50      0.42      0.46       880
          df       0.00      0.00      0.00        93
         mel       0.42      0.26      0.32       891
          nv       0.81      0.93      0.87      5365
        vasc       0.63      0.37      0.46       114

    accuracy                           0.73      8017
   macro avg       0.46      0.38      0.41      8017
weighted avg       0.69      0.73      0.70      8017





Evaluation report on Test data: 

              precision    recall  f1-score   support

       akiec       0.75      0.32      0.45        65
         bcc       0.47      0.65      0.55       102
         bkl       0.69      0.44      0.54       219
